# Download Satellite images
Note, this file is best run from google colab. To run smoothly, ensure that the package satimg_utils is in the same directory as this script and that the present notebook is stored within the current working directory.

In [3]:
import ee
import pandas as pd
import numpy as np
from typing import Dict, Tuple, Any

In [4]:
from satimg_utils.gee_classes import *
from satimg_utils.gee_helpers import *
from satimg_utils.gee_exporters import *

In [ ]:
# # mount the google drive
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/master_thesis/

In [5]:
ee.Authenticate()

Enter verification code: 4/1Adeu5BWpMkwQHTABUo_ZJwWoXuZxKk2CRqgGdrRnk5VHivsOBDbOZqrDiH8

Successfully saved authorization token.


In [6]:
ee.Initialize()

# Load LSMS Data

In [11]:
source_dir = '../../'
LSMS_CSV_PATH = f'{source_dir}/Data/lsms/processed/labels_cluster_v1.csv'
rs_raw_dir = f"{source_dir}/Data/satellite_imgs/RS_v2/RS_v2_raw/"
wp_raw_dir = f"{source_dir}/Data/satellite_imgs/RS_v2/WP_raw/"

lsms_df = pd.read_csv(LSMS_CSV_PATH, float_precision='high')
lsms_df = lsms_df[['country', 'start_ts', 'lat', 'lon', 'series', 'cluster_id', 'unique_id']].copy()
lsms_df['RS_file_name'] = 'RS_v2_'+lsms_df['unique_id'] + ".tif"
lsms_df['WP_file_name'] = 'WP_'+lsms_df['unique_id'] + ".tif"

# Download Remote Sensing images

In [8]:
# Download RS images
rs_downloaded_files = check_downloaded_files(prefix = "RS", source_dir = rs_raw_dir)
rs_mask = ~np.array([i in rs_downloaded_files for i in lsms_df['RS_file_name']])

to_download_df = lsms_df.loc[rs_mask,:].copy().reset_index(drop = True)
print(len(to_download_df))

0


In [9]:
lsms_surveys = list(to_download_df.groupby(['country', 'series', 'start_ts', 'cluster_id', 'unique_id']).groups.keys())

tasks = {}
tab_feats = []
count = 0

for country, series, start_ts, cluster_id, unique_id in tqdm(lsms_surveys):
    new_tasks, feats = export_rs_images(
        df=lsms_df, country=country,
        series = series, start_ts = start_ts,
        unique_id = unique_id)
    tasks.update(new_tasks)
    count += 1
    if count == 2999:
      print(f'Going to sleep for {60*30} seconds')
      time.sleep(60*30)
      count = 0

0it [00:00, ?it/s]

In [10]:
wait_on_tasks(tasks)

0it [00:00, ?it/s]

# Download WorldPop Data

In [12]:
# Download RS images
wp_downloaded_files = check_downloaded_files(prefix = "WP", source_dir = wp_raw_dir)
wp_mask = ~np.array([i in wp_downloaded_files for i in lsms_df['WP_file_name']])

to_download_df = lsms_df.loc[wp_mask,:].copy().reset_index(drop = True)
print(len(to_download_df))

0


In [13]:
lsms_surveys = list(to_download_df.groupby(['country', 'series', 'start_ts', 'cluster_id', 'unique_id']).groups.keys())

tasks = {}
count = 0

for country, series, start_ts, cluster_id, unique_id in tqdm(lsms_surveys):
    new_tasks = export_wp_imgs(
        df=lsms_df, country=country,
        series = series, start_ts = start_ts,
        unique_id = unique_id)
    tasks.update(new_tasks)
    count += 1
    if count == 2999:
      print(f'Going to sleep for {60*30} seconds')
      #time.sleep(60*30)
      count = 0

0it [00:00, ?it/s]

# Download Precipiation data

In [ ]:
lsms_surveys = list(lsms_df.groupby(['country', 'start_ts', 'unique_id', 'lat', 'lon']).groups.keys())

weather_feats = {key: None for key in lsms_df.unique_id}

for country, start_ts, unique_id, lat, lon in tqdm(lsms_surveys):
    subset_df = lsms_df[(lsms_df['unique_id'] == unique_id)].reset_index(drop = True)
    fcPoint = df_to_fc(subset_df)

    # get the RS derived datasets
    rs = RS_Feats(fcPoint.geometry(), start_ts = start_ts)

    precip_data = rs.init_precip().data
    weather_feats[unique_id] = precip_data

In [ ]:
weather_df = pd.DataFrame({
    'unique_id': weather_feats.keys(),
    'precipitation': weather_feats.values()
    })
weather_df.to_csv("Data/precipitation.csv", index = False)

# Download WSF data

In [5]:
cluster_df = lsms_df[['country', 'cluster_id', 'lat', 'lon']].drop_duplicates().reset_index(drop = True).copy()
lsms_surveys = list(cluster_df.groupby(['country', 'cluster_id']).groups.keys())

In [ ]:
# for country, cluster_id in tqdm(lsms_surveys):
#     subset_df = cluster_df[(cluster_df['cluster_id'] == cluster_id)].reset_index(drop = True).copy()
#     fcPoint = df_to_fc(subset_df)

#     # get the RS derived datasets
#     rs = RS_Feats(fcPoint.geometry(), start_ts = '2011-01-01')
#     break

In [6]:
tasks = {}

for country, cluster_id in tqdm(lsms_surveys):
    new_tasks = export_wsf_images(
        df = cluster_df, 
        country = country,
        cluster_id = cluster_id)
    tasks.update(new_tasks)

  0%|          | 0/2255 [00:00<?, ?it/s]

In [7]:
wait_on_tasks(tasks)

  0%|          | 0/2255 [00:00<?, ?it/s]

Task ('WSF_raw', 'eth', 'WSF_eth_010101088801601') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010102088801403') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010103010100106') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010103088801804') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010105088800204') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010106088800402') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010107010100116') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010201088800207') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010202020100107') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010202088800802') finished in 0 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_010203088801001') finished in 0 min with state: COMPLETED

Task ('WSF_raw', 'eth', 'WSF_eth_030309010100308') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030310088801701') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030403088801401') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030406088800801') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030409088801203') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030411088800903') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030412010100105') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030414088802101') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030416088800204') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030418010100309') finished in 4 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_030418010100807') finished in 5 min with state: COMPLETED

Task ('WSF_raw', 'eth', 'WSF_eth_040712088802302') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040713010100209') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040806088800802') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040808010100107') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040812088802401') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040817088802801') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040825010100102') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040901088800304') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040907088800105') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040909010100105') finished in 8 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_040910088801104') finished in 9 min with state: COMPLETED

Task ('WSF_raw', 'eth', 'WSF_eth_070103088800404') finished in 11 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070105088802502') finished in 11 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070108088800701') finished in 11 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070110010100102') finished in 11 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070111088800401') finished in 12 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070112088802605') finished in 12 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070115010100108') finished in 12 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070202010100105') finished in 12 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070202088801001') finished in 12 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070205088801302') finished in 12 min with state: COMPLETED
Task ('WSF_raw', 'eth', 'WSF_eth_070207088801001') finished in 12 min with state

...

Task ('WSF_raw', 'tza', 'WSF_tza_54-01-071-01-001') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-01-192-01-001') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-01-201-01-002') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-01-312-03-001') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-02-041-01-001') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-02-083-07-303') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-02-131-05-002') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54-02-241-02-001') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_5405303') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_5407002') finished in 75 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_54101002') finished in 75 min with state: COMPLETED
Tas

Task ('WSF_raw', 'tza', 'WSF_tza_7210047') finished in 78 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7210074') finished in 78 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7220058') finished in 78 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7301075') finished in 78 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7302004') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7303001') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7303002') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7307049') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7310003') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7310188') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7312048') finished in 79 min with state: COMPLETED
Task ('WSF_raw', 'tza', 'WSF_tza_7313053') finished in 79 min with state: CO

Task ('WSF_raw', 'uga', 'WSF_uga_1020028') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1020029') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1020030') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1020031') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1020032') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1020033') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1020034') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1030003') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1030005') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1040002') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1040004') finished in 83 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_1040006') finished in 83 min with state: CO

Task ('WSF_raw', 'uga', 'WSF_uga_2040015') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050001') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050004') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050009') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050013') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050017') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050022') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050026') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2050030') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2060001') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2060003') finished in 87 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_2070005') finished in 87 min with state: CO

Task ('WSF_raw', 'uga', 'WSF_uga_3070018') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3070020') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3070022') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3070023') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3070025') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3080001') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3080002') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3080004') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3090001') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3090002') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3100001') finished in 91 min with state: COMPLETED
Task ('WSF_raw', 'uga', 'WSF_uga_3100004') finished in 91 min with state: CO